# Создание датасета обучения

Данный документ предназначен для обработки и анализа предобработанных данных и построении пригодных для обучения данных

In [0]:
import pandas as pd
import numpy as np
import os
import gc
import seaborn as sns

from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/Shared drives/Кредитные риски')

In [0]:
from CreditRisks.metrics_library.rosstat_utils import *

In [0]:
DIR='Датасеты/'
DIR_IN=DIR+'Росстат_2019_компактный/'
DIR_OUT=DIR+'revision_005/'

# Генерация фич

In [0]:
for year in tqdm_notebook(range(YEAR_FIRST, YEAR_LAST+1)):
    D = pd.read_csv(f'{DIR_IN}{year}.csv', index_col='inn')
    D['okved2'] = D['okved'].str.extract(r'(^[0-9]+.[0-9]+)').fillna('__null__')
    D['okved1'] = D['okved'].str.extract(r'(^[0-9]+)').fillna('__null__')  
    D.to_csv(f'{DIR_OUT}01_src/{year}.csv')
    del D
    gc.collect()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Выбор релевантных компаний

In [0]:
companies = pd.read_csv(f'{DIR_IN}companies_info.csv', index_col="inn")

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
companies = companies[companies['okfs'] == 16]
companies.drop(columns=['okfs'], inplace=True)

In [0]:
if 'companies' in vars():
    companies.to_csv(f'{DIR_OUT}companies_relevant.csv', index_label='inn')
else:
    companies = pd.read_csv(f'{DIR_OUT}companies_relevant.csv', index_col='inn')

# Создание датасета существования компаний

In [0]:
def is_default(x):
    '''
    Расчитывает рейтинг компании
    9 - Актив не равен пассиву или 0 или заполнено менее 5 - неправильная отчетность
    1-2 - ok
    3-5 - дефолт
    '''
    cond_def = ((x['16003'] != x['17003']) | (x['16004'] != x['17004']) | (x['16004'] == 0) | (x['17004'] == 0) | (x['non_zero'] < 5)).astype(np.uint8)
    cond_a = ((x['12003'] != 0) & (x['15003'] != 0) & (x['12003']/x['15003'] < 0.5)).astype(np.uint8)
    cond_b = (x['16004']/x['16003'] > 2).astype(np.uint8)
    cond_c = ((x['12303'] != 0) & (x['12304'] != 0) & ((x['12303']/x['16003'])/(x['12004']/x['16004']) > 1)).astype(np.uint8)
    cond_d = ((x['12103'] != 0) & (x['12104'] !=0 ) & ((x['12103']/x['12104'] > 3) | (x['12104']/x['12103'] > 3))).astype(np.uint8)

    # оптимизированный else
    # +1 для "хорошей" компании
    return cond_def * 9 + (1 - cond_def) * (cond_a + cond_b + cond_c + cond_d + 1)

In [0]:
# Drop everything (keep index)
companies.drop(columns=['name','okpo','okopf','okved', 'region'], inplace=True)

In [0]:
for year in range(YEAR_FIRST, YEAR_LAST+1):
    D = pd.read_csv(f'{DIR_IN}{year}.csv', index_col='inn')
    # 4 - before COLUMNS_VALUE
    D.insert(4, 'non_zero', df[COLUMNS_VALUE].astype(bool).sum(axis=1))
    companies[f'{year}'] = pd.Series(0, index=companies.index, dtype=np.uint8, )
    total = 0
    for index, value in is_default(D).iteritems():
        if index in companies.index:
            companies.at[index, f'{year}'] = int(value)
            total += 1
    print(f"Done {year}, companies add {total}, total {D.shape[0]}")

Done 2012, companies add 699724, total 765769


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Done 2013, companies add 1609634, total 1732554
Done 2014, companies add 1815987, total 1957801
Done 2015, companies add 2174713, total 2333713
Done 2016, companies add 2202797, total 2364369
Done 2017, companies add 2202752, total 2361527
Done 2018, companies add 2126245, total 2282457


In [0]:
companies.head(100)

,2012,2013,2014,2015,2016,2017,2018
inn,,,,,,,
0,0,0,9,0,0,0,0
77,0,1,0,0,0,0,0
77202515,0,1,0,0,0,0,0
77247544,0,1,0,0,0,0,0
77250486,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
101006351,1,1,3,0,0,0,0
101006369,2,1,1,0,0,0,0
101006376,1,1,0,0,0,0,0


In [0]:
if 'companies' in vars():
    companies.to_csv(f'{DIR_OUT}companies_status.csv', index_label='inn')
else:
    dtypes = {}
    for year in range(YEAR_FIRST, YEAR_LAST+1):
        dtypes[str(year)] = np.uint8
    companies = pd.read_csv(f'{DIR_OUT}companies_status.csv', index_col='inn', dtype=dtypes)

# Генерация данных для обучения

## Генерация датасета истории

In [0]:
res = ((companies[f'{YEAR_FIRST}'] != 0) & (companies[f'{YEAR_FIRST}'] != 9)).astype(np.uint8)
for year in range(YEAR_FIRST+1, YEAR_LAST+1):
    res += ((companies[f'{year}'] != 0) & (companies[f'{year}'] != 9)).astype(np.uint8)
print('Before', companies.shape[0])
# Убираем компании, которые существуют менее 2 лет
companies.drop(labels=res[res < 2].index, inplace=True)
res.drop(labels=res[res < 2].index, inplace=True)
print('Drop fast dead', companies.shape[0])
# Новые компании, по которым мало информации
companies.drop(labels=res[(res < 3) & (companies[f'{YEAR_LAST}'] != 0) & (companies[f'{YEAR_LAST}'] != 9)].index, inplace=True)
print('Drop new', companies.shape[0])

Before 3682584
Drop fast dead 2249024
Drop new 2030083


In [0]:
drop_index = []
for index, row in companies.iterrows():
    exist = row[f'{YEAR_FIRST}'] != 0
    die = False
    for year in range(YEAR_FIRST+1, YEAR_LAST+1):
        exist_now = (row[str(year)] != 0) & (row[str(year)] != 9)
        if die and exist_now:
            drop_index.append(index)
            break
        if exist and not exist_now:
            die = True
        if not exist and exist_now:
            exist = True
        

In [0]:
companies.drop(labels=drop_index, inplace=True)
print('Drop companies without reports or with too bad reports', companies.shape[0])

Drop companies without reports or with too bad reports 1749787


In [0]:
if 'companies' in vars():
    companies.to_csv(f'{DIR_OUT}companies_status_filtered.csv', index_label='inn')
else:
    dtypes = {}
    for year in range(YEAR_FIRST, YEAR_LAST+1):
        dtypes[str(year)] = np.uint8
    companies = pd.read_csv(f'{DIR_OUT}companies_status_filtered.csv', index_col='inn', dtype=dtypes)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Генерация датасетов запроса

In [0]:
def generate_request_dataset(companies, year_begin, year_end, print_stats=True):
    # Удаляем компании, которые когда-либо подали плохую отчетность
    companies_ok = companies[(companies == 9).sum(axis=1) == 0]
    data = companies_ok.drop(columns=companies.columns)
    data['year_-1'] = -1
    data['year_0'] = -1
    data['target'] = -1
    for index, row in companies_ok.iterrows():
        # С компанией что-то не хорошо
        year = year_end
        data.at[index, 'target'] = ((row[f'{year}'] != 1) & (row[f'{year}'] != 2)).astype(np.uint8)
        year -= 1
        write = 0
        while year >= year_begin and write >= NUM_WRITE:
            if row[f'{year}'] != 0:
                data.at[index, f'year_{write}'] = year
                write -= 1
            year -= 1
    # Удаляем те записи, для которых не нашлось информации
    data.drop(index=data[(data == -1).sum(axis=1) > 0].index, inplace=True)
    print("Mean target: ", data['target'].mean(), "\tcount: ", len(data))
    gc.collect()
    return data

In [0]:
train = generate_request_dataset(companies, YEAR_FIRST, YEAR_LAST-1 )
test = generate_request_dataset(companies, YEAR_LAST-2, YEAR_LAST )
prod = generate_request_dataset(companies, YEAR_FIRST, YEAR_LAST )

Mean target:  0.333616619544029 	count:  1029583
Mean target:  0.17013850286597773 	count:  773558
Mean target:  0.4064442278780168 	count:  1081526


## Добавление категориальных признаков

In [0]:
companies_info = pd.read_csv(f'{DIR_IN}companies_info.csv', index_col='inn')
# okpo - идентификатор без смысла
# name - название - смысла мало
# okopf - организационно-правовая форма, сохраняется в 90% случаев
# okfs - форма собственности, сохраняется в 99% случаев
# okved - вид деятельности, сохраняется в 70% случаев
# inn - первые 2 цифры выведены в код региона
companies_info.drop(columns=['okpo', 'name', 'okopf', 'okfs', 'okved'], inplace=True)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
train = train.join(companies_info)
test = test.join(companies_info)
prod = prod.join(companies_info)

In [0]:
train.to_csv(f'{DIR_OUT}companies_request_train.csv', index_label='inn')
test.to_csv(f'{DIR_OUT}companies_request_test.csv', index_label='inn')
prod.to_csv(f'{DIR_OUT}companies_request_prod.csv', index_label='inn')

## Генерация датасета обучения

In [0]:
import csv
from typing import Union, List


class CsvMultiReader:
    __files = None
    __readers = None
    header = None
    __last_value = None

    def __init__(self, year_first: int, year_last: int, fmt: str):
        """
        Create multiply csv file reader for indexed data
        :param year_first: first file to read
        :param year_last: last file to read
        :param fmt: file path format string
        """
        self.__files = {}
        self.__readers = {}
        self.__last_value = {}
        for year in range(year_first, year_last + 1):
            self.__files[year] = open(fmt.format(year))
            self.__readers[year] = csv.reader(self.__files[year])
            row = self.__readers[year].__next__()
            if self.header is None:
                self.header = row
            else:
                assert row == self.header
            self.__last_value[year] = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        for k, f in self.__files.items():
            f.close()

    def get_index(self, year: int, index: int) -> Union[List, None]:
        """
        Read file until line with index is not appear
        :param year: year file to read
        :param index: index for search
        :return: certain row if exists, else None
        """
        row = self.__last_value[year]
        try:
            while row is None or int(row[0]) < index:
                row = next(self.__readers[year])
        except StopIteration:
            return None
        self.__last_value[year] = row
        if int(row[0]) == index:
            return row
        else:
            return None

In [0]:
tasks = ['train', 'test', 'prod', ]

In [0]:
for task in tasks:
    print("Task: ", task)
    fin = open(f'{DIR_OUT}02_middle/companies_request_{task}.csv', newline='')
    fout = open(f'{DIR_OUT}03_out/companies_ready_{task}.csv', 'w', newline='')

    rin = csv.reader(fin)
    rout = csv.writer(fout)
    head = rin.__next__()
    print(head)

    reader = CsvMultiReader(YEAR_FIRST, YEAR_LAST, f'{DIR_OUT}01_src/{{}}.csv')
    for h in reader.header[1:]:
        head.append(f'{head[1]}_{h}')
    for h in reader.header[1:]:
        head.append(f'{head[2]}_{h}')
    rout.writerow(head)

    with tqdm_notebook(total=10**6) as pbar:
        for row in rin:
            inn, y0, y1 = map(int, row[:3])
            r0 = reader.get_index(y0, inn)
            r1 = reader.get_index(y1, inn)
            if r0 is None or r1 is None:
                print(inn, 'is None')
            else:
                rout.writerow(row + r0[1:] + r1[1:])
            pbar.update()
            
    fin.close()
    fout.close()
    reader.__exit__(0,0,0)
    print("DONE!\n")

Task:  train
['inn', 'year_-1', 'year_0', 'target', 'region']


DONE!

Task:  test
['inn', 'year_-1', 'year_0', 'target', 'region']


DONE!

Task:  prod
['inn', 'year_-1', 'year_0', 'target', 'region']


DONE!



# Сохранение в формате pickle

In [0]:
NEW_DTYPES = RESULT_DTYPES.copy()
for key, dtype in NEW_DTYPES.items():
    try:
        _, _, c = key.split('_')
        code = int(c)
    except:
        continue
    NEW_DTYPES[key] = np.float32
for year in [0, -1]:
    for extra in [1, 2]:
        NEW_DTYPES[f'year_{year}_okved{extra}'] = str  

In [0]:
map_rules = []
for key, dtype in NEW_DTYPES.items():
    try:
        _, lag, c = key.split('_')
        lag = int(lag)
        code = int(c)
        old_code = f"year_-1_{code - 1}"
        new_code = f"year_0_{code + 5}"
        if lag == 0 and code % 10 == 4 and old_code in NEW_DTYPES:
            if new_code in NEW_DTYPES:
                print("Ahtyng!")
                print(key, old_code, new_code)
            map_rules.append((key, old_code, new_code))
    except ValueError:
        continue

In [0]:
for task in tasks:
    df = pd.read_csv(f'{DIR_OUT}03_out/companies_ready_{task}.csv', dtype=NEW_DTYPES)
    for col1, col2, col_new in map_rules:
        df[col_new] = (df[col1] * (df[col1] != 0) + df[col2] * (df[col1] == 0)).astype(np.float32)
        # print(col1, (df[col1] == df[col2]).mean())
    df.drop(columns=[col1, col2], inplace=True)
    df.to_pickle(f'{DIR_OUT}companies_ready_float_{task}.pkl')
    # print("="*80)